In [2]:
!pip install --upgrade watson-developer-cloud
!pip install --upgrade watson-machine-learning-client

Requirement already up-to-date: watson-developer-cloud in c:\users\user\anaconda3\envs\datascienceac\lib\site-packages (2.10.1)
Requirement already up-to-date: watson-machine-learning-client in c:\users\user\anaconda3\envs\datascienceac\lib\site-packages (1.0.378)


In [3]:
a =  1
a

1

### Configurando la API de watson
En este paso van a precisar usar el api_key que se generan en la página de IBM 

In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Eso aparecía a en el notebook que se bajan de la página (ya no funciona):
#from watson_developer_cloud import NaturalLanguageUnderstandingV1
#from watson_developer_cloud.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions

# Esta es la nueva forma de hacerlo
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, EmotionOptions, SentimentOptions

In [5]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('3x2RFmKUkalLtlgGJYOYewqbsWP-t4LcvvDfBgvLXL2l')
natural_language_understanding = NaturalLanguageUnderstandingV1(
   authenticator=authenticator,
    version='2019-07-25'
)
natural_language_understanding.set_service_url('https://gateway.watsonplatform.net/natural-language-understanding/api')

from ibm_watson import MyService
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('{apikey}')
service = MyService(
   authenticator=authenticator
)
service.set_service_url('{url}')

### Probando a watson
Exploremos el tipo de objeto que nos devuelve watson y como funciona para distintas frases

In [6]:
# Texto objetivo
text = 'This is a very silly text, it drives me crazy when I read it.'

# Importamos el modelo que vamos a usar y le avisamos que features queremo que devuelva
# y en que lenguaje está el texto (esto es mejor que dejarlo adivinar)
response = natural_language_understanding.analyze(text=text,features=Features(emotion=EmotionOptions(), sentiment=SentimentOptions()),language='en')
resultado = response.result
resultado

{'usage': {'text_units': 1, 'text_characters': 61, 'features': 2},
 'sentiment': {'document': {'score': -0.781525, 'label': 'negative'}},
 'language': 'en',
 'emotion': {'document': {'emotion': {'sadness': 0.155847,
    'joy': 0.14858,
    'fear': 0.318921,
    'disgust': 0.126755,
    'anger': 0.494942}}}}

Para tomar solo alguno de los resultados, podemos acceder al diccionario

In [7]:
print(resultado['sentiment']['document']['score'])

-0.781525


Probamos ahora un texto en español

In [8]:
# Texto objetivo
text = 'Este texto no tiene ningún objetivo, vive en la soledad que genera saber que su existencia no tiene un proposito determinado.'

# Importamos el modelo que vamos a usar y le avisamos que features queremo que devuelva
# y en que lenguaje está el texto (esto es mejor que dejarlo adivinar)
response = natural_language_understanding.analyze(text=text,features=Features(emotion=EmotionOptions(), sentiment=SentimentOptions()),language='es')
resultado = response.result
resultado


{'usage': {'text_units': 1, 'text_characters': 125, 'features': 1},
 'sentiment': {'document': {'score': 0, 'label': 'neutral'}},
 'language': 'es',
 'warnings': ['emotion: unsupported text language: es']}

### Dataset de Noticias Infobae de hoy
Este es dataset se lo preparamos para que puedan explorar como funciona Watson en textos en español. Algunas de las funcionalidades se pierden (no reporta las emociones), pero si clasifica como positivo o negativo los cuerpos de texto. Noten que otra estrategia podría ser traducir el texto y leugo aplicarle el algoritmo de watson en ingles. Esta otra estrategia no siempre funciona bien, depende mucho de la calidad de la traducción.

In [9]:
nombre_archivo = '2019_08_21.feather'

Exploremos el dataset

In [11]:
# Quiero ver el sentimiento de los cuerpos de la noticia.
noticias_hoy = pd.read_feather(nombre_archivo)
noticias_hoy

,Fecha,Id-Fecha,Subtitulo,Tema,Texto,Titulo
0,2019/08/21,0,"El tipo de cambio cerró con leve baja, pero lo...",economia,Hernán Lacunza logró sortear una prueba compl...,Hernán Lacunza consiguió una tregua con el dól...
1,2019/08/21,1,El encuentro será a las 11. Cuál puede ser el ...,politica,Hoy a las 11 de la mañana se verán las caras ...,Los diputados de Cambiemos se reunirán por pri...
2,2019/08/21,2,"En 1974, el ex juez Jorge Vicente Quiroga fue ...",sociedad,El domingo 28 de abril de 1974 Jorge Vicente ...,El asesinato del juez Quiroga: sabía que estab...
3,2019/08/21,3,"""Llaman todos"", aseguran los allegados al cand...",economia,"""Acá llaman todos, todo el mundo quiere venir...","Los empresarios en su laberinto: dudas, enojos..."
4,2019/08/21,4,"Hacienda mostró una vez más el rol de ""otros i...",economia,"Unas de las preocupaciones de ""los mercados"" ...",Las ganancias del Banco Central permitirán cum...
5,2019/08/21,5,Osvaldo Agosto era integrante de la Juventud P...,sociedad,"Sus ojos claros, vivaces, y esa sonrisa entre...","""Yo robé el sable corvo de San Martín"": el rel..."
6,2019/08/21,6,Gustavo Figueroa fue separado del cargo y some...,sociedad,"Gustavo Figueroa, quien prestaba servicios de...",Tucumán: trabajó durante nueve años como médic...
7,2019/08/21,7,El artista mexicano participó en Buenos Aires ...,sociedad,Eduardo Verástegui ya filmaba en Hollywood y ...,"La asombrosa conversión de Eduardo Verástegui,..."
8,2019/08/21,8,En las imágenes se ve a Jorge Martín Gómez par...,sociedad,Uno de los principales temas que dominó la co...,El video que muestra la amenaza del hombre que...
9,2019/08/21,9,La gobernadora bonaerense dejará de recibir 4....,politica,María Eugenia Vidal decidió postergar todos l...,María Eugenia Vidal postergó los anuncios econ...


In [12]:
cuerpos = noticias_hoy.Texto
titulos = noticias_hoy.Titulo
print(len(cuerpos))

24


Vamos a calcular el sentimiento para cada una de las noticias y agregarlo al dataframe 

In [13]:
score_cuerpos = []

for este_texto in cuerpos:
    
    # Quitamos del texto este símbolo que indica final de parrafo
    este_texto = este_texto.replace('\n','')
    
    # Analisis sentimiento del cuerpo
    # Le pedimos solo el sentimiento, y le avisamos que está en español
    response = natural_language_understanding.analyze(text=este_texto,features=Features(sentiment=SentimentOptions()),language='es')
    resultado = response.result
    score_cuerpos.append(resultado['sentiment']['document']['score'])

In [14]:
print(score_cuerpos)

[-0.628192, 0.703094, -0.603858, -0.361676, 0.357864, -0.413688, -0.851592, 0.304766, -0.740305, 0, -0.607553, 0.276372, 0, -0.621652, -0.490642, -0.40268, -0.85545, 0, 0.583955, -0.642936, 0.348123, -0.517949, -0.416298, -0.488772]


In [15]:
# Finalmente agregamos la columna al dataset
noticias_hoy['Sentimiento'] = score_cuerpos
noticias_hoy

,Fecha,Id-Fecha,Subtitulo,Tema,Texto,Titulo,Sentimiento
0,2019/08/21,0,"El tipo de cambio cerró con leve baja, pero lo...",economia,Hernán Lacunza logró sortear una prueba compl...,Hernán Lacunza consiguió una tregua con el dól...,-0.628192
1,2019/08/21,1,El encuentro será a las 11. Cuál puede ser el ...,politica,Hoy a las 11 de la mañana se verán las caras ...,Los diputados de Cambiemos se reunirán por pri...,0.703094
2,2019/08/21,2,"En 1974, el ex juez Jorge Vicente Quiroga fue ...",sociedad,El domingo 28 de abril de 1974 Jorge Vicente ...,El asesinato del juez Quiroga: sabía que estab...,-0.603858
3,2019/08/21,3,"""Llaman todos"", aseguran los allegados al cand...",economia,"""Acá llaman todos, todo el mundo quiere venir...","Los empresarios en su laberinto: dudas, enojos...",-0.361676
4,2019/08/21,4,"Hacienda mostró una vez más el rol de ""otros i...",economia,"Unas de las preocupaciones de ""los mercados"" ...",Las ganancias del Banco Central permitirán cum...,0.357864
5,2019/08/21,5,Osvaldo Agosto era integrante de la Juventud P...,sociedad,"Sus ojos claros, vivaces, y esa sonrisa entre...","""Yo robé el sable corvo de San Martín"": el rel...",-0.413688
6,2019/08/21,6,Gustavo Figueroa fue separado del cargo y some...,sociedad,"Gustavo Figueroa, quien prestaba servicios de...",Tucumán: trabajó durante nueve años como médic...,-0.851592
7,2019/08/21,7,El artista mexicano participó en Buenos Aires ...,sociedad,Eduardo Verástegui ya filmaba en Hollywood y ...,"La asombrosa conversión de Eduardo Verástegui,...",0.304766
8,2019/08/21,8,En las imágenes se ve a Jorge Martín Gómez par...,sociedad,Uno de los principales temas que dominó la co...,El video que muestra la amenaza del hombre que...,-0.740305
9,2019/08/21,9,La gobernadora bonaerense dejará de recibir 4....,politica,María Eugenia Vidal decidió postergar todos l...,María Eugenia Vidal postergó los anuncios econ...,0.000000


Exploremos un poco los resultados. Veamos el promedio de sentimiento de todas las noticias (esto nos va a dar una idea del ánimo general del día). Luego busquemos la noticia mas positiva y la mas negativa, para ver si coincidimos con el criterio.

In [16]:
todos_los_valores = noticias_hoy['Sentimiento']
animo_general = np.mean(todos_los_valores)
print(animo_general)

-0.252877875


In [17]:
noticia_positiva = noticias_hoy.loc[noticias_hoy['Sentimiento'].idxmax()]
print(noticia_positiva.Titulo)
print(noticia_positiva.Sentimiento)
#print(noticia_positiva.Subtitulo)
print(noticia_positiva.Texto)

Los diputados de Cambiemos se reunirán por primera vez después de la derrota en las PASO para expresar su respaldo al Gobierno
0.703094
 Hoy a las 11 de la mañana se verán las caras entre 70 y 80 diputados de Cambiemos, de los 107 que tiene actualmente el interbloque que conforman el PRO, el radicalismo y la Coalición Cívica (CC). Será la primera reunión del interbloque después de la derrota del oficialismo en las PASO. "La idea es hacer público el apoyo al Gobierno", le dijo una fuente parlamentaria a Infobae. 
  Además de evaluar el escenario político de acá a las elecciones generales de octubre, el interrogante sobre la supervivencia de la bancada oficialista es una de las preguntas que, por lo bajo, se hacen varios de los legisladores que responden a la coalición gobernante. 
  Los jefes de los bloques de los tres partidos que integran Cambiemos afirmaron ante Infobae que, ante una posible derrota del oficialismo en octubre, el interbloque parlamentario buscará mantenerse unido. Po

In [ ]:
noticia_negativa = noticias_hoy.loc[noticias_hoy['Sentimiento'].idxmin()]
print(noticia_negativa.Titulo)
print(noticia_negativa.Sentimiento)
#print(noticia_negativa.Subtitulo)
#print(noticia_negativa.Texto)

### Extra:

Explorar otros recursos disponibles en IBM cloud: https://cloud.ibm.com/catalog?search=label%3Alite

A continuación les dejamos un ejemplo de un modelo entrenado para clasificar comida.

In [18]:
import json
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('eKmTwbs_ByrOzP4eTDY3dc0VR502vK5zCIxVAp7-sQd5')
visual_recognition = VisualRecognitionV3(
    version='2018-03-19',
    authenticator=authenticator
)

visual_recognition.set_service_url('https://gateway.watsonplatform.net/visual-recognition/api')

url = 'https://ibm.biz/Bd2NPs'
classifier_ids = ["food"]

classes_result = visual_recognition.classify(url=url, classifier_ids=classifier_ids).get_result()
print(json.dumps(classes_result, indent=2))

{
  "images": [
    {
      "classifiers": [
        {
          "classifier_id": "food",
          "name": "food",
          "classes": [
            {
              "class": "apple",
              "score": 0.572,
              "type_hierarchy": "/fruit/accessory fruit/apple"
            },
            {
              "class": "accessory fruit",
              "score": 0.572
            },
            {
              "class": "fruit",
              "score": 0.805
            },
            {
              "class": "banana",
              "score": 0.5,
              "type_hierarchy": "/fruit/banana"
            }
          ]
        }
      ],
      "source_url": "https://ibm.biz/Bd2NPs",
      "resolved_url": "https://watson-developer-cloud.github.io/doc-tutorial-downloads/visual-recognition/fruitbowl.jpg"
    }
  ],
  "images_processed": 1,
  "custom_classes": 0
}


In [19]:
import json
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


url = 'https://watson-developer-cloud.github.io/doc-tutorial-downloads/visual-recognition/640px-IBM_VGA_90X8941_on_PS55.jpg'
classifier_ids = ["food"]

classes_result = visual_recognition.classify(url=url, classifier_ids=classifier_ids).get_result()
print(json.dumps(classes_result, indent=2))


{
  "images": [
    {
      "classifiers": [
        {
          "classifier_id": "food",
          "name": "food",
          "classes": [
            {
              "class": "non-food",
              "score": 1
            }
          ]
        }
      ],
      "source_url": "https://watson-developer-cloud.github.io/doc-tutorial-downloads/visual-recognition/640px-IBM_VGA_90X8941_on_PS55.jpg",
      "resolved_url": "https://watson-developer-cloud.github.io/doc-tutorial-downloads/visual-recognition/640px-IBM_VGA_90X8941_on_PS55.jpg"
    }
  ],
  "images_processed": 1,
  "custom_classes": 0
}
